In [21]:
!pip install fbprophet

import datetime
import io
import requests

from fbprophet import Prophet
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
from statsmodels.tsa.arima_model import ARIMA
from tqdm.notebook import tqdm

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [22]:
df = pd.read_csv('btc(1).csv', index_col=0)
df

,volume,volume weighted,open price,close price,high price,low price,timestamp,number
0,1853.726683,58263.4399,58049.000000,58632.00,58710.000000,57800.00,2021-03-20 00:00:00,31574
1,671.137374,58658.3421,58636.000000,58735.64,58887.410000,58491.92,2021-03-20 01:00:00,21873
2,1171.039574,58668.4997,58751.000000,58551.42,58916.010000,58421.60,2021-03-20 02:00:00,20182
3,828.101230,58385.3884,58551.420000,58390.00,58615.000000,58187.37,2021-03-20 03:00:00,19970
4,943.265974,58396.8700,58373.211671,58231.97,58631.940000,58153.00,2021-03-20 04:00:00,18637
...,...,...,...,...,...,...,...,...
2340,639.819480,32778.6041,32725.920000,32861.00,32989.000000,32576.90,2021-06-27 19:00:00,13085
2341,630.101561,32788.5058,32823.500000,32633.23,32983.000000,32620.00,2021-06-27 20:00:00,12818
2342,1153.339324,32609.5134,32635.440000,32862.40,32885.218314,32425.77,2021-06-27 21:00:00,16810
2343,5395.979243,34002.6519,32869.000000,34422.00,34639.360000,32820.25,2021-06-27 22:00:00,56107


In [23]:
fig = px.line(df, x="timestamp", y="close price", title='Close price change over time')
fig.show()

In [29]:
# test is 20 percent of the data
train_n = int(df.shape[0] * 0.8)
train = df[:train_n].copy()
test = df[train_n:].copy()

In [76]:
models = {
    'ARMA': {'order': (5,0,2)}, 
    'ARIMA': {'order': (6,1,3)},
    'Prophet': {'growth': 'linear', 'changepoint_prior_scale': 0.7}}

In [77]:
def ph_method(mdl_params, train, test):

  print('Time Series Forecasting: Prophet')
  ph = Prophet(**mdl_params)

  phdf = train[['timestamp', 'close price']]
  phdf.columns = ['ds', 'y']

  ph.fit(phdf)

  future = ph.make_future_dataframe(periods=len(test), freq='H', include_history=False)

  pred = ph.predict(future)

  return pred['yhat']

In [78]:
def arma_method(mdl_params, train, test, interval_size=50, name='Unknown'):

  obs = []
  train_ext = train.copy()

  for k in tqdm(range(0, len(test), interval_size), desc=f'Time Series Forecasting: {name}'):

    mdl = ARIMA(train_ext['close price'], **mdl_params)
    fmdl = mdl.fit()

    if k + interval_size > len(test):
      output = fmdl.forecast(len(test) - k)
      addit = test[k:].copy()
    else:
      addit = test[k:k+interval_size].copy()
      output = fmdl.forecast(interval_size)

    obs += list(output[0])
    train_ext = train_ext.append(addit).reset_index(drop=True)
  return obs

In [79]:
results = {}
for key, mdl_params in models.items():
  if 'AR' in key and 'MA' in key:
    res = arma_method(mdl_params, train, test, name=key)
  else:
    res = ph_method(mdl_params, train, test)

  fig = go.Figure()
  fig.add_traces(go.Scatter(x=df['timestamp'], y=df['close price'], name='Actual'))
  fig.add_traces(go.Scatter(x=test['timestamp'], y=res, name='Prediction'))
  fig.update_layout(
    title=f"{key}: Actual VS Prediction",
    xaxis_title="timestamp",
    yaxis_title="close price",
)
  fig.show()
  results[key] = res

Time Series Forecasting: ARMA:   0%|          | 0/10 [00:00<?, ?it/s]

Time Series Forecasting: ARIMA:   0%|          | 0/10 [00:00<?, ?it/s]

INFO:fbprophet:Disabling yearly seasonality. Run prophet with yearly_seasonality=True to override this.


Time Series Forecasting: Prophet


In [70]:
def get_profit(prices, ac_prices=None):
    if not ac_prices:
      ac_prices = prices[:]
    profit = 0
    for i in range(1, len(prices)):
        if prices[i] > prices[i-1]:
            profit += ac_prices[i] - ac_prices[i-1]
    return profit

In [126]:
def get_profit_2(price, ac_price):
    profit = 0
    buys = []
    sells = []
    n = len(price)
    if (n == 1):
        return
    i = 0
    while (i < (n - 1)):
        while ((i < (n - 1)) and
                (price[i + 1] <= price[i] * 1.0005)):
            i += 1
        if (i == n - 1):
            break
        buy = i
        i += 1
        while ((i < n) and (price[i] * 1.0005 >= price[i - 1])):
            i += 1
        sell = i - 1
        buys.append(buy)
        sells.append(sell)
        profit += (ac_price[sell] - ac_price[buy])

    return [profit, buys, sells]

In [133]:
def smape(A, F):
  return 100/len(A) * np.sum(np.abs(F - A) / (np.abs(A) + np.abs(F)))

mp = get_profit(list(test['close price']))

print(f'Max profit:{mp}')

closes = list(test['close price'])
for key, vals in results.items():
  metric = smape(test['close price'], np.array(vals))
  prof, buys, sells = get_profit_2(vals, list(test['close price']))
  fig = go.Figure()
  fig.add_traces(go.Scatter(
      x=df['timestamp'],
      y=df['close price'],
      name='close price'
))

  fig.add_trace(go.Scatter(
  x=[list(test['timestamp'])[k] for k in buys],
  y=[closes[k] for k in buys],
  marker=dict(color="green"),
  mode="markers",
  marker_symbol='triangle-up',
  marker_size=10,
  name='buy'
))
  
  fig.add_trace(go.Scatter(
  x=[list(test['timestamp'])[k] for k in sells],
  y=[closes[k] for k in sells],
  marker=dict(color="red"),
  mode="markers",
  marker_symbol='triangle-down',
  marker_size=10,
  name='sell'
))
  fig.show()
  print(f'{key} - SMAPE:{metric:.2f}%, PROFIT:{prof}')

Max profit:64070.18475861003


ARMA - SMAPE:2.14%, PROFIT:3164.1229072999995


ARIMA - SMAPE:2.03%, PROFIT:-2661.977092699999


Prophet - SMAPE:2.90%, PROFIT:-5662.810000000005
